In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
data_false=pd.read_excel(r'C:\Users\kerie\Scripts\Машинное обучение\model\Отказы.xlsx')
data_true=pd.read_excel(r'C:\Users\kerie\Scripts\Машинное обучение\model\Сделки.xlsx')

In [3]:
data_false=data_false.rename(columns={"Статус ТЛМ задания":"Статус_ТЛМ"})
data_false=data_false.query('ВыручкаОтПродаж >= 0 and Статус_ТЛМ == "5. Отказ"')

In [4]:
data_true = data_true[~data_true['СистемаНалогообложения'].isnull()]

In [5]:
data_false.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31641 entries, 0 to 46214
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Контрагент              31641 non-null  object        
 1   Код                     31641 non-null  int64         
 2   ИНН                     31641 non-null  int64         
 3   Наименование            31641 non-null  object        
 4   ВидДеятельности         31641 non-null  object        
 5   СистемаНалогообложения  31641 non-null  object        
 6   ВыручкаОтПродаж         31641 non-null  int64         
 7   КодОКВЭД                4980 non-null   object        
 8   Адрес                   19021 non-null  object        
 9   РегионОбслуживания      31641 non-null  object        
 10  Статус                  31641 non-null  object        
 11  НаправлениеБизнеса      31641 non-null  object        
 12  ДатаСозданияТЛМ         31641 non-null  dateti

- 31641 ТЛМ заданий с результатом ОТКАЗ, по нарпавлению бизнеса "2Б.Что делать 1С";
- Данные без пустых значений в столбцах СистемаНалогообложения, ВыручкаОтПродаж.

In [6]:
data_true.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1767 entries, 0 to 3122
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   КодТО                       1767 non-null   int64  
 1   Контрагент                  1767 non-null   object 
 2   ВидДеятельности             1767 non-null   object 
 3   Выручка 
Компании, руб      1767 non-null   int64  
 4   2021, руб                   1500 non-null   float64
 5   2022, руб                   1573 non-null   float64
 6   Вид купленной номенклатуры  1523 non-null   object 
 7   СистемаНалогообложения      1767 non-null   object 
 8   Адрес                       1733 non-null   object 
 9   РегионОбслуживания          1767 non-null   object 
 10  Статус                      1767 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 165.7+ KB


- 1767 ТЛМ заданий, по которым прошла сделка, по нарпавлению бизнеса "2Б.Что делать 1С";
- Данные без пустых значений в столбцах СистемаНалогообложения, ВыручкаОтПродаж.

Для обчения моделей будем использовать следующие параметры:
- СистемаНалогообложения;
- ВыручкаОтПродаж;
- РегионОбслуживания.
- Статус

# Кластеризируем данные со статусом ТЛМ задания "ОТКАЗ", для создания групп

Для кластеризации будем использовать метод, DBSCAN.

Для информации: 

В данном примере мы используем метод DBSCAN для кластеризации исходного датасета. Параметр `eps` задает максимальное расстояние между соседними точками, чтобы они считались одним кластером. Параметр `min_samples` определяет минимальное количество соседних точек, необходимое для формирования кластера.

Обратите внимание, что DBSCAN может определить количество кластеров автоматически, исходя из структуры данных.

In [7]:
data_false=data_false[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус']]
data_false.head()

,СистемаНалогообложения,ВыручкаОтПродаж,РегионОбслуживания,Статус
0,ОСНО,4147256000,Москва\МО,Действующее
2,УСНО 15%,85053000,Москва\МО,Действующее
3,ОСНО,203295000,Москва\МО,Действующее
5,ОСНО,26793000,Москва\МО,Действующее
8,ОСНО,75500000,Москва\МО,Действующее


In [8]:
#Номинативные переменные, меняем на числовые
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

category=['СистемаНалогообложения','РегионОбслуживания','Статус']

for i in category:
    le.fit(data_false[i])
    data_false[i+'_int']=le.transform(data_false[i])

data_false.head()

,СистемаНалогообложения,ВыручкаОтПродаж,РегионОбслуживания,Статус,СистемаНалогообложения_int,РегионОбслуживания_int,Статус_int
0,ОСНО,4147256000,Москва\МО,Действующее,4,1,0
2,УСНО 15%,85053000,Москва\МО,Действующее,7,1,0
3,ОСНО,203295000,Москва\МО,Действующее,4,1,0
5,ОСНО,26793000,Москва\МО,Действующее,4,1,0
8,ОСНО,75500000,Москва\МО,Действующее,4,1,0


# Стандартизация данных

In [9]:
from sklearn.preprocessing import StandardScaler
x=data_false[['ВыручкаОтПродаж','СистемаНалогообложения_int','РегионОбслуживания_int','Статус_int']]
x=np.nan_to_num(x)
scaler= StandardScaler().fit_transform(x) 

# Кластеризируем данные

In [10]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.3, min_samples=5)

dbscan.fit(scaler)

labels = dbscan.labels_

In [11]:
data_false['kluster']=labels
print(max(labels)+1,'')

28 


# Выводы:
- Метод DBSCAN разбил выборку на 28 кластеров;
- значения кластера с меткой -1, это выброс (не стандартные значения) таких получилось 49;

In [12]:
#data_false[data_false['kluster']==-1]
# Сохраняю отказы, разбитые по кластерам в файлик
data_false.to_excel("кластеризация_отказов.xlsx")

# Для обучения модели, отберем значения "Отказов" из каждого кластера, чтобы получилось 1720 значений, и 1720 значений сделок.

In [13]:
data_f=pd.concat([data_false[data_false['kluster']==27][:6], 
                  data_false[data_false['kluster']==26][:10],
                  data_false[data_false['kluster']==25][:10],
                  data_false[data_false['kluster']==24][:9],
                  data_false[data_false['kluster']==23][:7],
                  data_false[data_false['kluster']==22][:4],
                  data_false[data_false['kluster']==21][:8],
                  data_false[data_false['kluster']==20][:14],
                  data_false[data_false['kluster']==19][:60],
                  data_false[data_false['kluster']==18][:70],
                  data_false[data_false['kluster']==17][:10],
                  data_false[data_false['kluster']==16][:48],
                  data_false[data_false['kluster']==15][:5],
                  data_false[data_false['kluster']==14][:20],
                  data_false[data_false['kluster']==13][:23],
                  data_false[data_false['kluster']==12][:39],
                  data_false[data_false['kluster']==11][:15],
                  data_false[data_false['kluster']==10].sample(n=130),
                  data_false[data_false['kluster']==9].sample(n=130),
                  data_false[data_false['kluster']==8].sample(n=130),
                  data_false[data_false['kluster']==7].sample(n=130),
                  data_false[data_false['kluster']==6].sample(n=130),
                  data_false[data_false['kluster']==5].sample(n=130),
                  data_false[data_false['kluster']==4][:62],
                  data_false[data_false['kluster']==3].sample(n=130),
                  data_false[data_false['kluster']==2].sample(n=130),
                  data_false[data_false['kluster']==1].sample(n=130),
                  data_false[data_false['kluster']==0].sample(n=130)])

# Отказы
data_f['Сделка']=0
data_f=data_f[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус','Сделка']]

# Сделки
data_t=data_true[:1720]. \
rename(columns={"Выручка \nКомпании, руб":"ВыручкаОтПродаж"})
data_t['Сделка']=1
data_t=data_t[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус','Сделка']]

data=pd.concat([data_f,data_t])

#Номинативные переменные, меняем на числовые
le=LabelEncoder()
category=['СистемаНалогообложения','РегионОбслуживания','Статус']

for i in category:
    le.fit(data[i])
    data[i+'_int']=le.transform(data[i])

# перемешаем Датасет
data = data.sample(frac=1)
data.head(5)

,СистемаНалогообложения,ВыручкаОтПродаж,РегионОбслуживания,Статус,Сделка,СистемаНалогообложения_int,РегионОбслуживания_int,Статус_int
18837,ОСНО,234636000,Другой регион,Недействующие,0,4,0,1
19171,УСНО 15%,0,Другой регион,Действующее,0,7,0,0
755,ОСНО,3994230000,Москва\МО,Действующее,1,4,1,0
32338,"ОСНО,УСНО",123359000,Москва\МО,Действующее,0,5,1,0
1183,УСНО 6%,2316000,Москва\МО,Действующее,1,8,1,0


# Данные для прогноза

In [14]:
data_f_p=pd.concat([data_false[data_false['kluster']==27][6:],
                    data_false[data_false['kluster']==26][10:],
                    data_false[data_false['kluster']==25][10:],
                    data_false[data_false['kluster']==24][9:],
                    data_false[data_false['kluster']==23][7:],
                    data_false[data_false['kluster']==22][4:],
                    data_false[data_false['kluster']==21][8:],
                    data_false[data_false['kluster']==20][14:],
                    data_false[data_false['kluster']==19][60:],
                    data_false[data_false['kluster']==18][72:],
                    data_false[data_false['kluster']==17][10:],
                    data_false[data_false['kluster']==16][48:],
                    data_false[data_false['kluster']==15][5:],
                    data_false[data_false['kluster']==14][20:],
                    data_false[data_false['kluster']==13][23:],
                    data_false[data_false['kluster']==12][39:],
                    data_false[data_false['kluster']==11][:15],
                    data_false[data_false['kluster']==10].sample(n=2),
                    data_false[data_false['kluster']==9].sample(n=2),
                    data_false[data_false['kluster']==8].sample(n=2),
                    data_false[data_false['kluster']==7].sample(n=2),
                    data_false[data_false['kluster']==6].sample(n=2),
                    data_false[data_false['kluster']==5].sample(n=2),
                    data_false[data_false['kluster']==4][62:],
                    data_false[data_false['kluster']==3].sample(n=2),
                    data_false[data_false['kluster']==2].sample(n=2),
                    data_false[data_false['kluster']==1].sample(n=2),
                    data_false[data_false['kluster']==0].sample(n=2)])


data_f_p['Сделка']=0
data_f_p=data_f_p[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус','Сделка']]

data_t_p=data_true[1720:]. \
rename(columns={"Выручка \nКомпании, руб":"ВыручкаОтПродаж"})
data_t_p['Сделка']=1
data_t_p=data_t_p[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус','Сделка']]

data_p=pd.concat([data_t_p,data_f_p])

data_p=data_p[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус','Сделка']]

data_p.to_excel("Данные для прогноза.xlsx",index=False)

In [15]:
# Номинативные переменные, заменяем на качественные, данными, как при обучении
sn=['Аутсорсинг','Бюджет','ЕНВД','ЕСХН','ОСНО','ОСНО,УСНО','Патент','УСНО 15%','УСНО 6%']
def alert(row):
    for i in range(0,9):
         if row['СистемаНалогообложения']==sn[i]:
                return i
data_p['СистемаНалогообложения']=data_p.apply(alert, axis=1) 

ro=['Другой регион','Москва\МО']
def alert_r(row):
    for i in range(0,2):
         if row['РегионОбслуживания']==ro[i]:
                return i
data_p['РегионОбслуживания']=data_p.apply(alert_r, axis=1)

st=['Действующее','Недействующие','Нет информации']
def alert_s(row):
    for i in range(0,3):
         if row['Статус']==st[i]:
                return i
data_p['Статус']=data_p.apply(alert_s, axis=1)

data_p = data_p.sample(frac=1)

In [16]:
data_p.rename(columns={"СистемаНалогообложения":"СистемаНалогообложения_int",
                              "РегионОбслуживания":"РегионОбслуживания_int",
                              "Статус":"Статус_int"}, inplace = True) 
# перемешаем Датасет
data_p.head(10)

,СистемаНалогообложения_int,ВыручкаОтПродаж,РегионОбслуживания_int,Статус_int,Сделка
3029,7,129704000,1,0,1
3115,7,117221000,1,0,1
45958,1,463007000,1,0,0
3119,8,13041000,1,0,1
3059,7,8518000,1,0,1
3077,7,17242000,1,0,1
3111,4,195214000,1,0,1
3028,4,44633000,1,0,1
14362,7,7875000,0,0,0
3078,7,5880000,1,0,1


# Модель на CatBoost

In [17]:
from catboost import CatBoostClassifier
from catboost import Pool
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [18]:
data.shape

(3440, 8)

In [19]:
#Отбор признаков
# Будем перебирать все комбинации атрибутов
from itertools import combinations
name_columns=list(data[['СистемаНалогообложения_int','ВыручкаОтПродаж','РегионОбслуживания_int','Статус_int',]].columns)
feature = sum([list(map(list, combinations(name_columns, i))) for i in range(len(name_columns) + 1)], [])
feature.pop(0)
feature

[['СистемаНалогообложения_int'],
 ['ВыручкаОтПродаж'],
 ['РегионОбслуживания_int'],
 ['Статус_int'],
 ['СистемаНалогообложения_int', 'ВыручкаОтПродаж'],
 ['СистемаНалогообложения_int', 'РегионОбслуживания_int'],
 ['СистемаНалогообложения_int', 'Статус_int'],
 ['ВыручкаОтПродаж', 'РегионОбслуживания_int'],
 ['ВыручкаОтПродаж', 'Статус_int'],
 ['РегионОбслуживания_int', 'Статус_int'],
 ['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'РегионОбслуживания_int'],
 ['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'Статус_int'],
 ['СистемаНалогообложения_int', 'РегионОбслуживания_int', 'Статус_int'],
 ['ВыручкаОтПродаж', 'РегионОбслуживания_int', 'Статус_int'],
 ['СистемаНалогообложения_int',
  'ВыручкаОтПродаж',
  'РегионОбслуживания_int',
  'Статус_int']]

In [20]:
x_pred=data_p.iloc[:,:-1]
y_true=data_p.iloc[:,-1]

In [21]:
# Данные
x=data[['СистемаНалогообложения_int','ВыручкаОтПродаж','РегионОбслуживания_int','Статус_int',]] # Зависимые переменные 
y=data['Сделка'] # Прогноз

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20,random_state=42)


info_test=[]
info_p=[]

# будем перебирать все признаки 
# и смотреть метрику на тестовой выборке,
# и на выборке, которая не учавствует в обучении.
# Метрики для оценки качества Accuracy и ROK-AUC


for i in feature:
    train_pool=Pool(data=x_train[i], label=y_train)
    validation_pool=Pool(data=x_test[i],label=y_test)

    model = CatBoostClassifier(
    iterations=11500,
    learning_rate = 0.003,
    random_seed=63,
    custom_loss=['AUC', 'Accuracy'],
    use_best_model=True)
    
    
    model.fit(
    train_pool, 
    eval_set=validation_pool,
    verbose=False,
    plot=False);
    
    pred=model.predict(x_pred[i])
    pred_test=model.predict(x_test[i])
    
    info_p.append([i,round(metrics.accuracy_score(y_true,pred),2), round(metrics.roc_auc_score(y_true,pred),2)])
    info_test.append([i,round(metrics.accuracy_score(y_test,pred_test),2), round(metrics.roc_auc_score(y_test,pred_test),2)]) 

In [22]:
print('Результат на тестовой выборке, Accuracy and ROK-AUC')
info_test = sorted(info_test, key=lambda x: (x[1], -x[1]))
for i in info_test:
    print(i)

Результат на тестовой выборке, Accuracy and ROK-AUC
[['ВыручкаОтПродаж'], 0.54, 0.55]
[['РегионОбслуживания_int'], 0.61, 0.6]
[['Статус_int'], 0.62, 0.61]
[['ВыручкаОтПродаж', 'Статус_int'], 0.63, 0.62]
[['ВыручкаОтПродаж', 'РегионОбслуживания_int'], 0.65, 0.64]
[['РегионОбслуживания_int', 'Статус_int'], 0.71, 0.7]
[['СистемаНалогообложения_int'], 0.72, 0.71]
[['ВыручкаОтПродаж', 'РегионОбслуживания_int', 'Статус_int'], 0.72, 0.72]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж'], 0.74, 0.73]
[['СистемаНалогообложения_int', 'РегионОбслуживания_int'], 0.76, 0.75]
[['СистемаНалогообложения_int', 'Статус_int'], 0.78, 0.78]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'РегионОбслуживания_int'], 0.78, 0.77]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'Статус_int'], 0.79, 0.79]
[['СистемаНалогообложения_int', 'РегионОбслуживания_int', 'Статус_int'], 0.83, 0.83]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'РегионОбслуживания_int', 'Статус_int'], 0.85, 0.84]


In [23]:
print('Результат на прогнозной выборке, Accuracy and ROK-AUC')
info_p = sorted(info_p, key=lambda x: (x[1], -x[1]))
for i in info_p:
    print(i)

Результат на прогнозной выборке, Accuracy and ROK-AUC
[['ВыручкаОтПродаж'], 0.5, 0.49]
[['СистемаНалогообложения_int'], 0.52, 0.52]
[['СистемаНалогообложения_int', 'Статус_int'], 0.57, 0.57]
[['Статус_int'], 0.58, 0.61]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж'], 0.59, 0.59]
[['ВыручкаОтПродаж', 'Статус_int'], 0.64, 0.67]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'Статус_int'], 0.65, 0.66]
[['РегионОбслуживания_int'], 0.68, 0.71]
[['ВыручкаОтПродаж', 'РегионОбслуживания_int'], 0.74, 0.76]
[['СистемаНалогообложения_int', 'РегионОбслуживания_int'], 0.79, 0.8]
[['РегионОбслуживания_int', 'Статус_int'], 0.8, 0.82]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'РегионОбслуживания_int'], 0.85, 0.85]
[['ВыручкаОтПродаж', 'РегионОбслуживания_int', 'Статус_int'], 0.85, 0.86]
[['СистемаНалогообложения_int', 'РегионОбслуживания_int', 'Статус_int'], 0.88, 0.89]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'РегионОбслуживания_int', 'Статус_int'], 0.92, 0.93]


# Вывод:
- Лучше всего показала модель на тестовых и на прогнозных данных, с атрибутами 

СистемаНалогообложения_int, ВыручкаОтПродаж, РегионОбслуживания_int, Статус_int

In [24]:
# Оценка влияния фичей (признаков) в % соотношении
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,РегионОбслуживания_int,55.095896
1,СистемаНалогообложения_int,24.341235
2,Статус_int,12.872688
3,ВыручкаОтПродаж,7.690182


In [25]:
# Оценка влияния фичей (признаков) на loss функцию в % соотношении
model.get_feature_importance(
    validation_pool,   #LossFunctionChange
    prettified=True)

,Feature Id,Importances
0,РегионОбслуживания_int,50.736640
1,СистемаНалогообложения_int,27.759678
2,Статус_int,14.008664
3,ВыручкаОтПродаж,7.495018


In [26]:
print('Результат на прогнозной выборке, Accuracy and ROK-AUC')
print(info_p[-1])
print()
print('Результат на тестовой выборке, Accuracy and ROK-AUC')
print(info_test[-1])

Результат на прогнозной выборке, Accuracy and ROK-AUC
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'РегионОбслуживания_int', 'Статус_int'], 0.92, 0.93]

Результат на тестовой выборке, Accuracy and ROK-AUC
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'РегионОбслуживания_int', 'Статус_int'], 0.85, 0.84]


# Сохраняем модель

In [27]:
model.save_model("Catboost_Model_1C.bin")

# Итоговая модель с визуализацией

In [28]:
info_test_new=[]
info_p_new=[]

model_over = CatBoostClassifier(
         iterations=11500,
         learning_rate = 0.003,
         random_seed=63,
         custom_loss=['AUC', 'Accuracy'],
         use_best_model=True)
    
model_over.fit(
         train_pool, 
         eval_set=validation_pool,
         verbose=False,
         plot=True);
    
pred_new=model_over.predict(x_pred)
pred_test_new=model_over.predict(x_test)
    

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [29]:
info_p_new.append([round(metrics.accuracy_score(y_true,pred_new),2), 
                   round(metrics.roc_auc_score(y_true,pred_new),2)])

info_test_new.append([round(metrics.accuracy_score(y_test,pred_test_new),2), 
                      round(metrics.roc_auc_score(y_test,pred_test_new),2)])

In [30]:
print(info_test_new)

[[0.85, 0.84]]


In [31]:
print(info_p_new)

[[0.92, 0.93]]
